In [5]:
import pandas as pd
import os
import cv2
import zipfile
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report

# 1. Анализ и очистка датасета

In [3]:
!kaggle datasets download -d alexattia/the-simpsons-characters-dataset

Dataset URL: https://www.kaggle.com/datasets/alexattia/the-simpsons-characters-dataset
License(s): CC-BY-NC-SA-4.0
 99% 1.07G/1.08G [00:08<00:00, 69.6MB/s]
100% 1.08G/1.08G [00:08<00:00, 134MB/s] 


In [6]:
with zipfile.ZipFile('the-simpsons-characters-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [8]:
file_path = '/content/number_pic_char.csv'
df = pd.read_csv(file_path)
print(df)

    Unnamed: 0                      name  total  train  test  bounding_box
0            0             Homer Simpson   2246   1909   337           612
1            1              Ned Flanders   1454   1236   218           595
2            2               Moe Szyslak   1452   1234   218           215
3            3              Lisa Simpson   1354   1151   203           562
4            4              Bart Simpson   1342   1141   201           554
5            5             Marge Simpson   1291   1097   194           557
6            6          Krusty The Clown   1206   1025   181           226
7            7         Principal Skinner   1194   1015   179           506
8            8  Charles Montgomery Burns   1193   1014   179           650
9            9       Milhouse Van Houten   1079    917   162           210
10          10              Chief Wiggum    986    838   148           209
11          11    Abraham Grampa Simpson    913    776   137           595
12          12           

In [15]:
#Видимо, что для некоторых персонажей информации очень мало. Посмотрим, насколько мало
min_values = df.nsmallest(20, 'train')
print(min_values)

    Unnamed: 0                  name  total  train  test  bounding_box
42          42           Jimbo Jones      0      0     0             0
43          43         Bumblebee Man      0      0     0             0
44          44          Hans Moleman      0      0     0             0
45          45         Helen Lovejoy      0      0     0             0
46          46        Jasper Beardly      0      0     0             0
41          41           Lionel Hutz      3      3     0             0
39          39             Disco Stu      8      7     1             0
40          40          Troy Mcclure      8      7     1             0
38          38           Miss Hoover     17     14     3             0
36          36              Fat Tony     27     23     4             0
37          37                   Gil     27     23     4             0
35          35             Otto Mann     32     27     5             0
34          34          Sideshow Mel     40     34     6             0
33    

Google Cloud рекомендует для CNN классификатора минимум 50–100 изображений на класс.
50 - если изображения внутри класса похожи. Важно не потерять разнообразие классов, поэтому будем проводить аугментацию для всех персонажей.

In [11]:
# Исключаем классы, в которых менее 50 изображений
df_filtered = df[df['train'] >= 50]
print(df_filtered)

    Unnamed: 0                      name  total  train  test  bounding_box
0            0             Homer Simpson   2246   1909   337           612
1            1              Ned Flanders   1454   1236   218           595
2            2               Moe Szyslak   1452   1234   218           215
3            3              Lisa Simpson   1354   1151   203           562
4            4              Bart Simpson   1342   1141   201           554
5            5             Marge Simpson   1291   1097   194           557
6            6          Krusty The Clown   1206   1025   181           226
7            7         Principal Skinner   1194   1015   179           506
8            8  Charles Montgomery Burns   1193   1014   179           650
9            9       Milhouse Van Houten   1079    917   162           210
10          10              Chief Wiggum    986    838   148           209
11          11    Abraham Grampa Simpson    913    776   137           595
12          12           

In [12]:
# Вычисляем среднее количество изображений на класс
mean_count = df_filtered['train'].mean()
print(mean_count)

583.1333333333333


In [23]:
# Из распределения мы знаем, что у нас значительный дисбаланс классов
# Воспользуемся правилом х delta чтобы определить слишком малые и большие классы

delta = 6 #во сколько раз отличие от среднего

# Разбиваем классы на группы
# Слишком много данных (более чем в delta раз превышает среднее)
too_many_data = df_filtered[df_filtered['train'] > delta * mean_count]['name']

# Оптимальное количество данных (близко к среднему, в пределах delta)
optimal_data = df_filtered[(df_filtered['train'] <= delta * mean_count) & (df['train'] >= mean_count / delta)]['name']

# Слишком мало данных (более чем в delta раз меньше среднего)
too_few_data = df_filtered[df_filtered['train'] < mean_count / delta]['name']


# Преобразуем имена в нужный формат (чтобы совпадало с тем, что в папках)
formatted_too_many_data = too_many_data.str.lower().str.replace(' ', '_')
formatted_optimal_data = optimal_data.str.lower().str.replace(' ', '_')
formatted_too_few_data = too_few_data.str.lower().str.replace(' ', '_')

# Результаты
print("Классы с слишком большим количеством данных:", formatted_too_many_data.tolist())
print("Классы с оптимальным количеством данных:", formatted_optimal_data.tolist())
print("Классы с слишком малым количеством данных:", formatted_too_few_data.tolist())

Классы с слишком большим количеством данных: []
Классы с оптимальным количеством данных: ['homer_simpson', 'ned_flanders', 'moe_szyslak', 'lisa_simpson', 'bart_simpson', 'marge_simpson', 'krusty_the_clown', 'principal_skinner', 'charles_montgomery_burns', 'milhouse_van_houten', 'chief_wiggum', 'abraham_grampa_simpson', 'sideshow_bob', 'apu_nahasapeemapetilon', 'kent_brockman', 'comic_book_guy', 'edna_krabappel', 'nelson_muntz', 'lenny_leonard', 'mayor_quimby', 'waylon_smithers', 'maggie_simpson', 'groundskeeper_willie', 'barney_gumble', 'selma_bouvier', 'carl_carlson', 'ralph_wiggum']
Классы с слишком малым количеством данных: ['patty_bouvier', 'martin_prince', 'professor_john_frink']


<ipython-input-23-fe346e2fed1a>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  optimal_data = df_filtered[(df_filtered['train'] <= delta * mean_count) & (df['train'] >= mean_count / delta)]['name']


In [24]:
dataset_folder = 'simpsons_dataset'

In [26]:
# Преобразуем имена в нужный формат (чтобы совпадало с тем, что в папках)
formatted_too_many_data = too_many_data.str.lower().str.replace(' ', '_')
formatted_optimal_data = optimal_data.str.lower().str.replace(' ', '_')
formatted_too_few_data = too_few_data.str.lower().str.replace(' ', '_')

In [27]:
# Разделение на images и labels
images = []
labels = []

# Перебираем папки и собираем изображения и метки
for character in pd.concat([formatted_optimal_data, formatted_too_few_data]):
    character_folder_path = os.path.join(dataset_folder, character)
    if os.path.exists(character_folder_path):
        for file_name in os.listdir(character_folder_path):
            file_path = os.path.join(character_folder_path, file_name)
            if os.path.isfile(file_path):
                images.append(file_path)
                labels.append(character)

# Проверяем результаты разделения на images и labels
print(f'Количество изображений: {len(images)}')
print(f'Количество меток: {len(labels)}')
print(f'Пример изображения: {images[0] if images else "Нет изображений"}')
print(f'Пример метки: {labels[0] if labels else "Нет меток"}')

Количество изображений: 20582
Количество меток: 20582
Пример изображения: simpsons_dataset/homer_simpson/pic_1166.jpg
Пример метки: homer_simpson


# 3. Аугментация данных
Генерируем доп изображения в малые классы, чтобы скорректировать дисбаланс классов.

Что подойдет:
- Небольшое изменение масштаба, сдивиги, поворот


Что не подойдет:
- Ротация на 90 градусов и более
- Сильная обрезка изображений, фрагментирование портретов
- Изменение цвета для мультка со узнаваемой цветовой схемой это может быть фатально
- Сильный шум или размытие не характерны для скриншотов из мультика — такие методы скорее подойдут для фото из реальной жизни

Аугментацию можно проводить как отдельно, так и на лету (онлайн), прямо во время обучения модели, с использованием тех же библиотек tensorflow или аналогов.

Оставлю пример кода как для отедльной, так и для онлайновой аугментации.

In [28]:
# Настройка параметров генерации изображений
augmented_images = []
augmented_labels = []

datagen = ImageDataGenerator(
    rescale=1./255, #чтобы привести значение пикселя к от 0 до 1 (так лучше для модели)
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

augmentation_counts = {}  # Для отслеживания количества сгенерированных изображений для каждого класса
augmented_image_paths = []  # Для хранения путей к сгенерированным изображениям

augmented_dataset_folder = "augmented_dataset"  # Папка для сохранения сгенерированных изображений
os.makedirs(augmented_dataset_folder, exist_ok=True)

In [29]:
# Аугментация для классов с недостаточным количеством данных
for character in formatted_too_few_data:
    character_folder_path = os.path.join(dataset_folder, character)
    augmented_character_folder_path = os.path.join(augmented_dataset_folder, character)
    os.makedirs(augmented_character_folder_path, exist_ok=True)

    if os.path.exists(character_folder_path):
        current_count = len(os.listdir(character_folder_path))
        target_count = int(mean_count)  # Целевое количество изображений для каждого класса = среднее
        images_needed = target_count - current_count
        augmentation_counts[character] = 0

        if images_needed > 0:
            for file_name in os.listdir(character_folder_path):
                file_path = os.path.join(character_folder_path, file_name)
                if os.path.isfile(file_path):
                    # Проверка существования файла и корректности пути
                    if not os.path.exists(file_path):
                        print(f"Ошибка: файл {file_path} не существует.")
                        continue
                    try:
                        # Чтение изображения
                        img = cv2.imread(file_path)
                        if img is None:
                            print(f"Ошибка: не удалось прочитать изображение {file_path}.")
                            continue
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = cv2.resize(img, (64, 64))  # Приводим к единому размеру
                         # Применение аугментации
                        img = img.reshape((1,) + img.shape)  # Преобразуем для генератора
                        i = 0
                        for batch in datagen.flow(img, batch_size=1):
                            augmented_img = (batch[0] * 255).astype(np.uint8)  # Преобразуем обратно в изображение
                            save_path = os.path.join(augmented_character_folder_path, f"augmented_{file_name.split('.')[0]}_{i}.jpg")
                            cv2.imwrite(save_path, cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))
                            augmented_images.append(save_path)
                            augmented_labels.append(character)
                            augmentation_counts[character] += 1
                            augmented_image_paths.append(save_path)
                            i += 1
                            if i >= images_needed:  # Ограничение на количество сгенерированных изображений для достижения целевого количества
                                break
                        if augmentation_counts[character] >= images_needed:
                            break
                    except Exception as e:
                        print(f"Ошибка при обработке файла {file_path}: {e}")


In [30]:
# Таблица с количеством сгенерированных изображений для каждого класса
augmentation_df = pd.DataFrame(list(augmentation_counts.items()), columns=['Класс', 'Количество сгенерированных изображений'])
print(augmentation_df)

                  Класс  Количество сгенерированных изображений
0         patty_bouvier                                     306
1         martin_prince                                     307
2  professor_john_frink                                     313


In [31]:
# Теперь можно объединить оригинальные и аугментированные изображения и метки
images.extend(augmented_images)
labels.extend(augmented_labels)

print(f'Общее количество изображений после аугментации: {len(images)}')
print(f'Общее количество меток после аугментации: {len(labels)}')

Общее количество изображений после аугментации: 21508
Общее количество меток после аугментации: 21508


# 4. Создание модели

In [33]:
# Разделение на тренировочный и тестовый наборы
data = pd.DataFrame({'image': images, 'label': labels})
X_train, X_test, y_train, y_test = train_test_split(data['image'].tolist(), data['label'].tolist(), test_size=0.2, stratify=data['label'], random_state=42)

In [34]:
# Подготовка данных для модели
filtered_X_train = []
filtered_y_train = []
for img_path, label in zip(X_train, y_train):
    img = cv2.imread(str(img_path))
    if img is not None:
        img = cv2.resize(img, (64, 64)) #поискать другой трансформер для пред процессинга
        filtered_X_train.append(img)
        filtered_y_train.append(label)
X_train_images = np.array(filtered_X_train) / 255.0  # Нормализация

filtered_X_test = []
filtered_y_test = []
for img_path, label in zip(X_test, y_test):
    img = cv2.imread(str(img_path))
    if img is not None:
        img = cv2.resize(img, (64, 64))
        filtered_X_test.append(img)
        filtered_y_test.append(label)
X_test_images = np.array(filtered_X_test) / 255.0  # Нормализация


In [35]:
# Преобразование меток в категориальный вид
label_mapping = {label: idx for idx, label in enumerate(data['label'].unique())}
filtered_y_train = np.array([label_mapping[label] for label in filtered_y_train])
filtered_y_test = np.array([label_mapping[label] for label in filtered_y_test])
y_train = to_categorical(filtered_y_train, num_classes=len(label_mapping))
y_test = to_categorical(filtered_y_test, num_classes=len(label_mapping))

In [36]:
# Создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_mapping), activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Softmax — ф-я для преобразования вектора реальных чисел в вероятностное распределение. Подходит для задач многоклассовой классификации входных данных.

Преобразует каждый элемент входного вектора в значение от 0 до 1, причем сумма всех выходных значений равна 1. Это позволяет интерпретировать выходные данные как вероятности принадлежности к каждому из классов.

# 5. Компиляция и обучение модели

In [37]:
# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Категориальная кросс-энтропия — это функция потерь, используемая в задачах многоклассовой классификации, где модель должна предсказать вероятность принадлежности примера к одной из нескольких категорий.

Измеряет разницу между истинным распределением вероятностей (обычно закодированным в формате one-hot) и предсказанным распределением вероятностей, полученным от модели (обычно через softmax).

In [38]:
# Обучение модели
model.fit(X_train_images, y_train, validation_data=(X_test_images, y_test), epochs=10, batch_size=32)

Epoch 1/10
538/538 ━━━━━━━━━━━━━━━━━━━━ 96s 175ms/step - accuracy: 0.1943 - loss: 2.8680 - val_accuracy: 0.5132 - val_loss: 1.8032
Epoch 2/10
538/538 ━━━━━━━━━━━━━━━━━━━━ 144s 179ms/step - accuracy: 0.4642 - loss: 1.9172 - val_accuracy: 0.5979 - val_loss: 1.4625
Epoch 3/10
538/538 ━━━━━━━━━━━━━━━━━━━━ 142s 180ms/step - accuracy: 0.5589 - loss: 1.5264 - val_accuracy: 0.6574 - val_loss: 1.2492
Epoch 4/10
538/538 ━━━━━━━━━━━━━━━━━━━━ 140s 177ms/step - accuracy: 0.6378 - loss: 1.2523 - val_accuracy: 0.7073 - val_loss: 1.0793
Epoch 5/10
538/538 ━━━━━━━━━━━━━━━━━━━━ 96s 179ms/step - accuracy: 0.6942 - loss: 1.0356 - val_accuracy: 0.7197 - val_loss: 1.0233
Epoch 6/10
538/538 ━━━━━━━━━━━━━━━━━━━━ 94s 174ms/step - accuracy: 0.7349 - loss: 0.8687 - val_accuracy: 0.7262 - val_loss: 0.9854
Epoch 7/10
538/538 ━━━━━━━━━━━━━━━━━━━━ 142s 174ms/step - accuracy: 0.7729 - loss: 0.7321 - val_accuracy: 0.7417 - val_loss: 0.9480
Epoch 8/10
538/538 ━━━━━━━━━━━━━━━━━━━━ 143s 175ms/step - accuracy: 0.8006 - lo

In [40]:
# Оценка модели и метрики
y_pred = model.predict(X_test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Вывод отчета по метрикам классификации
report = classification_report(y_true_classes, y_pred_classes, target_names=list(label_mapping.keys()))
print(report)

135/135 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step
                          precision    recall  f1-score   support

           homer_simpson       0.64      0.77      0.70       449
            ned_flanders       0.77      0.84      0.80       291
             moe_szyslak       0.60      0.84      0.70       290
            lisa_simpson       0.63      0.54      0.58       271
            bart_simpson       0.57      0.59      0.58       268
           marge_simpson       0.88      0.90      0.89       258
        krusty_the_clown       0.84      0.90      0.87       241
       principal_skinner       0.78      0.76      0.77       239
charles_montgomery_burns       0.68      0.67      0.67       239
     milhouse_van_houten       0.83      0.86      0.85       216
            chief_wiggum       0.77      0.85      0.81       197
  abraham_grampa_simpson       0.83      0.68      0.75       183
            sideshow_bob       0.91      0.86      0.88       175
  apu_nahasapeemapetilon       0.

Precision — показывает, насколько точно модель предсказывает положительные классы. Если модель классифицирует кадры с Гомером Симпсоном, и она предсказывает 10 кадров как "Гомер", но только 8 из них действительно являются Гомером, то precision будет равен 80%.


Recall — это метрика, которая показывает, насколько хорошо модель находит все истинные положительные примеры. Если в наборе данных есть 15 кадров с Гомером, а модель правильно классифицировала только 8 из них, то recall будет равен 53% (8 истинных положительных из 15 фактических положительных).


F1-score — это гармоническое среднее между precision и recall. Он позволяет объединить обе метрики в одну и дает более полное представление о производительности модели. Использование F1-score полезно в ситуациях с несбалансированными классами, когда важно учитывать как точность, так и полноту.

Support — это просто количество истинных экземпляров для каждого класса в тестовом наборе данных. Показывает, сколько тестовых примеров есть в каждом классе и помогает понять распределение классов в данных.

Лосс (или функция потерь) — это мера, которая показывает, насколько хорошо или плохо модель предсказывает результаты по сравнению с фактическими значениями. Она используется для оценки качества модели во время обучения и оптимизации. Лосс помогает алгоритму понять, насколько далеко его предсказания от истинных значений, и на основании этого корректировать свои параметры. Используется в процессе обучения, тогда как метрики — оценивают его итог.